# Preprocessing pipeline

Objective of this notebook is to aggregate aggregate seismic processing actions into a neat pipeline. 

* [Dataset](#Dataset)
* [Pipeline](#Pipeline)
* [Conclusion](#Conclusion)
* [Suggestions for improvement](#Suggestions-for-improvement)

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from time import time

In [2]:
from seismicpro import FieldIndex, TraceIndex, SeismicBatch, SeismicDataset
from seismicpro.batchflow import Dataset, Pipeline, V, B, D, L, I

from seismicpro.src import (seismic_plot,
                            gain_plot, calculate_sdc_quality,
                            measure_gain_amplitude, merge_segy_files
                           )

## Dataset

For this task we make use of Dataset 1 for processing testing. It contains several records with various amplitude range:

In [3]:
path_raw = '/data/preproc/1_input_4_PREP_full_pipeln.sgy'
field_index = FieldIndex(name='raw', path=path_raw, extra_headers='all')
field_set = SeismicDataset(field_index, SeismicBatch)

field_index.head()

,GroupX,Correlated,ScalarTraceHeader,SourceEnergyDirectionMantissa,TRACE_SAMPLE_INTERVAL,GainType,GeophoneGroupNumberRoll1,SourceUpholeTime,LowCutFrequency,LowCutSlope,...,SweepTraceTaperLengthStart,SweepType,DataUse,OverTravel,SweepLength,SourceMeasurementUnit,SourceWaterDepth,NSummedTraces,TRACE_SEQUENCE_FILE,file_id
,,,,,,,,,,,...,,,,,,,,,raw,raw
FieldRecord,,,,,,,,,,,,,,,,,,,,,
8834,1496439,0,8435,-2147483648,2000,1,0,0,30,12,...,0,0,1,0,0,0,0,1,1,/data/preproc/1_input_4_PREP_full_pipeln.sgy
8834,1496589,0,8435,-2147483648,2000,1,0,0,30,12,...,0,0,1,0,0,0,0,1,2,/data/preproc/1_input_4_PREP_full_pipeln.sgy
8834,1496738,0,8435,-2147483648,2000,1,0,0,30,12,...,0,0,1,0,0,0,0,1,3,/data/preproc/1_input_4_PREP_full_pipeln.sgy
8834,1496888,0,4769,-2147483648,2000,1,0,0,30,12,...,0,0,1,0,0,0,0,1,4,/data/preproc/1_input_4_PREP_full_pipeln.sgy
8834,1497038,0,4769,-2147483648,2000,1,0,0,30,12,...,0,0,1,0,0,0,0,1,5,/data/preproc/1_input_4_PREP_full_pipeln.sgy


## Pipeline

For spherical divergence correction we'll use predefined speed values:

In [4]:
speed = np.array([1524]*700 + [1924.5]*300 + [2184.0]*400 +  [2339.6]*400 + 
                 [2676]*150 + [2889.5]*2250 + [3566]*2800 + [4785.3]*1000)

In this pipeline we preform spherical divergence correction (SDC) and remove traces that contain more than 50 consequent zero values, then save the results. Parameters for SDC are calculated via optimization procedure only once for all dataset in `before` instance of pipeline.

In [5]:
tmp_dump_path = '/data/preproc/tmp/'
output_path = '/data/preproc/processed/merged.sgy'

first_preproc_ppl = (
    field_set.pipeline()
    .init_variable('sdc_params')
    .load(fmt='sgy', components='raw')
    .correct_spherical_divergence(src='raw', dst='raw', speed=speed, params=V('sdc_params'))
    .drop_zero_traces(src='raw', dst='raw', num_zero=50)
    .dump(path=L(lambda x: os.path.join(tmp_dump_path, str(x) + '.sgy'))(I()),
          src='raw', fmt='segy', split=False)
    .run_later(batch_size=1, n_epochs=1, shuffle=False, drop_last=False, bar=True)
    )

(first_preproc_ppl.before
                  .find_sdc_params(components='raw', speed=speed, loss=calculate_sdc_quality, initial_point=(2, 1), save_to=V('sdc_params')))

Let's run it:

In [6]:
first_preproc_ppl.run()

100%|██████████| 30/30 [01:52<00:00,  1.86s/it]


In [7]:
output_path = '/data/preproc/processed/merged.sgy'
merge_segy_files(output_path=output_path, extra_headers='all',
                 path=os.path.join(tmp_dump_path, '*.sgy'))

100%|██████████| 30/30 [00:26<00:00,  1.78it/s]


We'll use results of the previous pipeline as input for the next one:

In [8]:
field_index = FieldIndex(name='raw', path=output_path, extra_headers='all')
field_set = SeismicDataset(field_index, SeismicBatch)

This pipeline calculated parameters for equalization in `before` instance and applies them to each batch in main pipeline:

In [9]:
output_path = '/data/preproc/processed/final.sgy'

second_preproc_ppl = (
    field_set.pipeline()
    .load(fmt='sgy', components='raw')
    .equalize(src='raw', dst='raw', params=V('equal_params'))
    .dump(path=L(lambda x: os.path.join(tmp_dump_path, str(x) + '.sgy'))(I()),
          src='raw', fmt='segy', split=False)
    .run_later(batch_size=1, n_epochs=1, shuffle=False, drop_last=False, bar=True)
)

(second_preproc_ppl.before
                   .find_equalization_params(component='raw', record_id='YearDataRecorded', save_to=V('equal_params')))

Run equalization:

In [10]:
second_preproc_ppl.run()

100%|██████████| 30/30 [00:44<00:00,  1.23s/it]


In [11]:
merge_segy_files(output_path=output_path, extra_headers='all',
                 path=os.path.join(tmp_dump_path, '*.sgy'))

100%|██████████| 30/30 [00:23<00:00,  1.75it/s]


## Conclusion

Current version of notebook contains preprocessing pipeline with three actions: correction for spherical divergence, removal of traces with more than 50 consecuent zeros and equalization of amplitudes.

## Suggestions for improvement

The next step in development of pipeline is adding two more actions: correction of inverted traces and attenuation of 50/60 Hz noise.

Another direction for improvement is unification of all actions in one whole pipeline.